In [1]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from app.utils.textpreprocessing import TextPreprocessor
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

In [7]:
textprepo = TextPreprocessor()

In [67]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='colbert-ir/colbertv2.0',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

No sentence-transformers model found with name colbert-ir/colbertv2.0. Creating a new one with MEAN pooling.


In [68]:
db_faiss = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/vector_database_creation/faiss_anime_index_colbert", embeddings = embedding_function)

In [69]:
import yaml

# Load API key from config.yaml
with open("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/app/config.yaml", "r") as file:
    config = yaml.safe_load(file)

api_key = config["api_key"]

In [70]:
import google.generativeai as genai
userdata = {"GOOGLE_API_KEY": api_key}
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [71]:

def filter_tokens(metadata):
        metadata_tokens = metadata.get("tokens", [])
        metadata_studio = metadata.get("studio", [])
        metadata_producer = metadata.get("producer", [])
        metadata_licensors = metadata.get("licensors", [])
        #metadata_cf_recs = metadata.get("cf_recs", [])
        #metadata_pop_recs = metadata.get("pop_recs", [])
        metadata_genre = metadata.get("genre", [])
        return any(token in metadata_tokens for token in query_token) or metadata["score"] > 5.0 or any(token in metadata_studio for token in query_token) or any(token in metadata_producer for token in query_token) or any(token in metadata_licensors for token in query_token) or any(token in metadata_genre for token in query_token) 


In [72]:
retriever = db_faiss.as_retriever(search_kwargs={"k": 50, "filter": filter_tokens})
#retriever = db_faiss.as_retriever()

In [73]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0,google_api_key = GOOGLE_API_KEY)

In [74]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """
```Instructions
You are an anime afficinado who went deep weeb under your mothers basement who knows all about anime and japanese culture. 
Use the following pieces of context to summarize different animes at the end.
Use only use stuff from the context to answer the answer and summarize the question. 
Recommend up to 5 anime and provide a one sentence description for each anime.
Always say "Thanks for using the anime recommender from Casual Correlations!" at the end of the answer.
```

Below is an example
```
Question: can you recommend me 5 animes from the 90

Summary: 

1. Neon Genesis Evangelion: A group of teenagers pilot giant robots to protect humanity from mysterious creatures known as Angels, in a series renowned for its complex characters and psychological themes.

2. Cowboy Bebop: This space-western series follows the adventures of a group of bounty hunters aboard the spaceship Bebop, with a jazzy soundtrack and stylish animation.

3. Dragon Ball Z: Goku and his friends defend Earth from powerful foes in this action-packed series that became a major phenomenon.

4. Sailor Moon: Usagi Tsukino transforms into the magical warrior Sailor Moon to battle evil forces and protect the world, known for its empowering themes and strong female characters.

5. Ghost in the Shell: Major Motoko Kusanagi and her team investigate cybercrimes in a future where humanity is interconnected with technology, exploring questions of identity and consciousness.

Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes from studio ghibli

1. My Neighbor Totoro (1988): Directed by Hayao Miyazaki, this heartwarming tale follows two young sisters who encounter friendly forest spirits in rural Japan.

2. Spirited Away (2001): Also directed by Hayao Miyazaki, Spirited Away tells the story of a young girl named Chihiro who becomes trapped in a mysterious and magical world, where she must work in a bathhouse for spirits to rescue her parents.

3. Princess Mononoke (1997): Directed by Hayao Miyazaki, this epic fantasy film explores the conflict between industrialization and nature, as a young prince becomes entangled in a struggle between forest gods and human settlers.

4. Howl's Moving Castle (2004): Directed by Hayao Miyazaki, this enchanting film follows a young woman named Sophie who is cursed by a witch and seeks refuge in the moving castle of the wizard Howl.

5. Kiki's Delivery Service (1989): Directed by Hayao Miyazaki, this charming film follows a young witch named Kiki who moves to a new town and starts a delivery service using her flying broomstick.

Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes similar to one piece

1. Fairy Tail: This series follows the adventures of Natsu Dragneel and his friends in the Fairy Tail guild as they take on various quests and battles in the magical land of Fiore. Like One Piece, it features a diverse cast of characters, epic battles, and a strong sense of camaraderie.

2. Naruto: Naruto follows the journey of Naruto Uzumaki, a young ninja with dreams of becoming the strongest ninja and leader of his village, the Hokage. It features a similar blend of action, humor, and heartfelt moments, as well as a focus on friendship and determination.

3. Hunter x Hunter: This series follows Gon Freecss, a young boy who aspires to become a Hunter like his father, as he embarks on various adventures and challenges in search of his father and his own identity. It shares themes of friendship, adventure, and personal growth with One Piece.

4. Bleach: Bleach follows Ichigo Kurosaki, a teenager with the ability to see ghosts, as he becomes a Soul Reaper and battles evil spirits known as Hollows. Like One Piece, it features intense battles, supernatural elements, and a large cast of characters.

5. One Punch Man: While it's more of a parody of traditional shonen anime, One Punch Man shares some similarities with One Piece in its action-packed battles and larger-than-life characters. It follows Saitama, a hero who can defeat any opponent with a single punch, as he seeks a worthy opponent and navigates the world of superheroes.

Thanks for using the anime recommender from Casual Correlations!

Question: I like the character like monkey d luffy what anime can you recommend other than one piece

1. Naruto: Naruto Uzumaki, the main character of this series, shares some similarities with Luffy. He's determined to become the strongest ninja and leader of his village, and he possesses a similar sense of optimism and loyalty to his friends.

2. Fairy Tail: Natsu Dragneel, the protagonist of Fairy Tail, is known for his adventurous spirit, boundless energy, and strong sense of loyalty to his friends, much like Luffy. The series is filled with exciting adventures and epic battles.

3. Hunter x Hunter: Gon Freecss, the main character of Hunter x Hunter, shares Luffy's sense of adventure and determination. Like Luffy, Gon is on a quest to achieve his goals and is willing to face any challenge that comes his way.

4. My Hero Academia: Izuku Midoriya, also known as Deku, shares Luffy's determination and unwavering spirit. Despite facing many obstacles, Deku never gives up on his dream of becoming a hero and protecting others.

5. Dragon Ball series (Dragon Ball, Dragon Ball Z, Dragon Ball Super): Goku, the main character of the Dragon Ball series, shares Luffy's adventurous nature and love of fighting. Like Luffy, Goku is always eager to challenge strong opponents and push his limits.

Thanks for using the anime recommender from Casual Correlations!
```

{context}

Question: {question}

Summary:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [75]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [76]:
#BlockedPromptException
#recommend popular recommendations 

In [77]:
def get_anime_recommendations(query):
    
    # Invoke the RAG chain
    streamlit_results = rag_chain.invoke(query)
    doc_results = retriever.get_relevant_documents(query)
    
    return streamlit_results, doc_results


In [78]:
# Example usage
query = 'can you recommend me anime for kids'
query_token = textprepo.preprocess_text(query)
recommendations, doc_results = get_anime_recommendations(query)
print(recommendations)  

1. Tsuredure Children: A series of short stories about the various romantic relationships between high school students.

2. Hanamaru Kindergarten: A group of kindergarteners try to win the attention of their caretaker, who is more interested in a fellow teacher.

3. School Babysitters: Two brothers help out at their school's daycare center while attending normal classes.

4. Little Lulu and Her Little Friends: The everyday adventures of a group of young children.

5. Momokuri: A teenage girl who is obsessed with her boyfriend tries to win his affection.

Thanks for using the anime recommender from Casual Correlations!


In [79]:
[(doc.metadata['anime_id'], doc.metadata['name']) for doc in doc_results]

[(34902, 'Tsurezure Children'),
 (6574, 'Hanamaru Youchien'),
 (16740, 'Hanamaru Youchien Special Preview'),
 (35222, 'Gakuen Babysitters'),
 (37959, 'Gakuen Babysitters Special'),
 (4903, 'Kyou no 5 no 2 (TV)'),
 (29825, 'Onnanoko tte.'),
 (44274, 'Puraore! Pride of Orange'),
 (39689, 'Nande Koko ni Sensei ga!? Nande Koko ni Sensei-tachi ga!?'),
 (38397, 'Nande Koko ni Sensei ga!?'),
 (9587, 'Oniichan no Koto nanka Zenzen Suki ja Nai n da kara ne!!'),
 (3823, 'Little Lulu to Chicchai Nakama'),
 (35427, 'Animegataris'),
 (24765, 'Gakkougurashi!'),
 (14289, 'Suki tte Ii na yo.'),
 (17409, 'Suki tte Ii na yo. Mei and Marshmallow'),
 (30014, 'Momokuri'),
 (41454, 'Eizouken Mini Anime'),
 (39792, 'Eizouken ni wa Te wo Dasu na!'),
 (34280, 'Gamers!')]